In [33]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Feb 9 2025

@author: Yaning
"""

from peft import LoraConfig, get_peft_model
import torch
from torch.utils.data import DataLoader, Dataset

In [49]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "dbmdz/german-gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [50]:
tokenizer.pad_token = tokenizer.eos_token

In [51]:
# Read the text file
with open('C:/Users/Administrator/Desktop/llm/winnetou_s.txt', 'r') as file:
    text_lines = file.readlines()

In [52]:
max_length = 512

In [53]:
# If the input text exceeds the max_length, split it into smaller chunks
def split_text_into_chunks(text, chunk_size):
    # Tokenize the entire text and split it into smaller chunks
    tokenized_text = tokenizer(text, padding=False, truncation=False, return_tensors="pt")
    total_tokens = tokenized_text['input_ids'][0]
    chunks = [total_tokens[i:i + chunk_size] for i in range(0, len(total_tokens), chunk_size)]
    return chunks

# Split into chunks (if necessary)
chunks = split_text_into_chunks(text_lines[0], max_length)


In [54]:
# Custom Dataset for single line split into chunks
class TextDataset(Dataset):
    def __init__(self, chunks):
        self.chunks = chunks

    def __len__(self):
        return len(self.chunks)

    def __getitem__(self, idx):
        return {
            "input_ids": torch.tensor(self.chunks[idx]),
            "attention_mask": torch.ones_like(self.chunks[idx]),  # Attention mask for the whole sequence
            "labels": torch.tensor(torch.cat((chunks[0][1:],torch.tensor([1])), dim=0))
        }

# Create a dataset instance
dataset = TextDataset(chunks)

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [55]:
# Configure LoRA settings
lora_config = LoraConfig(
    r=20,  # Rank of the low-rank matrices (can experiment with different values)
    lora_alpha=3,  # Scaling factor
    lora_dropout=0.3,  # Dropout rate for LoRA layers
    task_type="CAUSAL_LM"  # Causal Language Model task
)

# Apply LoRA to the model
lora_model = get_peft_model(model, lora_config)

In [56]:
from sklearn.model_selection import train_test_split

train_size = int(0.8 * len(dataset))
train_dataset, eval_dataset = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])


In [57]:
from transformers import DataCollatorForLanguageModeling

# Use the default data collator for language modeling tasks (this will handle padding)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Since GPT-2 is a causal language model
)


In [58]:
from transformers import Trainer, TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./german-gpt2-lora",
    eval_strategy="epoch",
    learning_rate=2e-3,
    per_device_train_batch_size=1,
    num_train_epochs=5,
    logging_dir="./logs",
    logging_strategy="epoch",  # Log at intervals
    logging_steps=1,  # Log loss every 10 steps
    # report_to="tensorboard",
    # save_steps=10_000,
    # save_total_limit=2,
)

# Assuming you have already set up the LoRA model (lora_model)
trainer = Trainer(
    model=lora_model,  # Your LoRA model
    args=training_args,
    train_dataset=train_dataset,  # Pass the dataset here
    eval_dataset = eval_dataset,
    data_collator=data_collator
)

# Start training
trainer.train()


In [34]:
model.save_pretrained('./german-gpt2-lora/full')
tokenizer.save_pretrained('./german-gpt2-lora/full')

('./german-gpt2-lora/full\\tokenizer_config.json',
 './german-gpt2-lora/full\\special_tokens_map.json',
 './german-gpt2-lora/full\\vocab.json',
 './german-gpt2-lora/full\\merges.txt',
 './german-gpt2-lora/full\\added_tokens.json',
 './german-gpt2-lora/full\\tokenizer.json')

In [35]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Replace with the directory where your model is saved
fine_tuned_model_dir = "./german-gpt2-lora/full"  # Adjust this if needed

# Load the fine-tuned model and tokenizer
model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_dir)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_dir)


Some weights of the model checkpoint at ./german-gpt2-lora/full were not used when initializing GPT2LMHeadModel: ['transformer.h.0.attn.c_attn.base_layer.bias', 'transformer.h.0.attn.c_attn.base_layer.weight', 'transformer.h.0.attn.c_attn.lora_A.default.weight', 'transformer.h.0.attn.c_attn.lora_B.default.weight', 'transformer.h.1.attn.c_attn.base_layer.bias', 'transformer.h.1.attn.c_attn.base_layer.weight', 'transformer.h.1.attn.c_attn.lora_A.default.weight', 'transformer.h.1.attn.c_attn.lora_B.default.weight', 'transformer.h.10.attn.c_attn.base_layer.bias', 'transformer.h.10.attn.c_attn.base_layer.weight', 'transformer.h.10.attn.c_attn.lora_A.default.weight', 'transformer.h.10.attn.c_attn.lora_B.default.weight', 'transformer.h.11.attn.c_attn.base_layer.bias', 'transformer.h.11.attn.c_attn.base_layer.weight', 'transformer.h.11.attn.c_attn.lora_A.default.weight', 'transformer.h.11.attn.c_attn.lora_B.default.weight', 'transformer.h.2.attn.c_attn.base_layer.bias', 'transformer.h.2.attn.c

In [63]:
input_text = "ÜbersetzÜbersetzen Sie die folgenden Sätze in den sächsischen Dialekt: Unser Team besteht aus mehr als 180 Personen, darunter renommierte internationale Forscher sowie hochqualifizierte Fachkräfte in administrativen und kommunikativen Rollen. Mit mehr als 60 leitenden Forschern, zwei Humboldt-Professuren und bis zu zwölf geplanten KI-Professuren unterstützen wir Exzellenz in der Forschung und Lehre in Leipzig und Dresden. Die Förderung von jungen Talenten ist ebenfalls ein wichtiger Bestandteil unserer Arbeit, weshalb wir vier Junior-Forschungsgruppen gegründet haben, die unsere aktuellen Forschungsthemen sinnvoll ergänzen. Darüber hinaus begrüßen wir assoziierte Mitglieder, die ihr Fachwissen in unser Zentrum einbringen."
# input_text = "Was ist die Liebe"
# input_text = "Schreiben Sie eine Kurzgeschichte über zwei Sachsen bei der Deutschen Bahn"

# Tokenize the input text
input_ids = tokenizer.encode(input_text, return_tensors="pt")

In [64]:
# Set generation parameters (you can customize these)
output = model.generate(
    input_ids,
    max_length=255,       # Maximum length of the generated text
    num_return_sequences=1,  # Number of sequences to return
    temperature=0.7,      # Control randomness in generation
    top_k=50,             # Limit to the top_k most likely next tokens
    top_p=0.95,           # Nucleus sampling (only use top_p probability mass)
    do_sample=True,       # Whether to use sampling (True) or greedy decoding (False)
)

# Decode the generated output back to text
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ÜbersetzÜbersetzen Sie die folgenden Sätze in den sächsischen Dialekt: Unser Team besteht aus mehr als 180 Personen, darunter renommierte internationale Forscher sowie hochqualifizierte Fachkräfte in administrativen und kommunikativen Rollen. Mit mehr als 60 leitenden Forschern, zwei Humboldt-Professuren und bis zu zwölf geplanten KI-Professuren unterstützen wir Exzellenz in der Forschung und Lehre in Leipzig und Dresden. Die Förderung von jungen Talenten ist ebenfalls ein wichtiger Bestandteil unserer Arbeit, weshalb wir vier Junior-Forschungsgruppen gegründet haben, die unsere aktuellen Forschungsthemen sinnvoll ergänzen. Darüber hinaus begrüßen wir assoziierte Mitglieder, die ihr Fachwissen in unser Zentrum einbringen. Die Forschungsgruppe ist in der Lage, auf den neuesten Stand der Forschung zu kommen, weil sie sich spezialisiert hat, um in ihrer Arbeit nicht zu stark mit den Universitäten verbunden zu sein, die an der Universität in der Wissenschaft nicht vertreten sind. Das ist e